In [2]:
import numpy as np

This evaluates the circuit 
<img src="./images/common_collector_amplifier.PNG" />

In [11]:
def eval_common_collector_amp_circuit(val, R_Source, R_Load):
    Vdc_th = val['Vcc'] * (val['R2']/(val['R1'] + val['R2']))
    Rb_th = 1.0 / (1.0/val['R1'] + 1.0/val['R2'])    

    #KVL for DC characteristics
    #Ie = (beta + 1)*Ib
    # Vdc_th - Ib*Rb_th - Vbe - Ie*Re = 0V
    # Vdc_th - Ib*Rb_th - Vbe - (beta + 1)*Ib*Re = 0V
    # Vdc_th - Vbe = Ib*Rb_th + (beta + 1)*Ib*Re
    # Ib * (Rb_th + (beta + 1)*Re) = Vdc_th - Vbe
    # Ib = (Vdc_th - Vbe)/(Rb_th + (beta + 1)*Re)
        
    Ib = (Vdc_th - val['Vbe']) / (Rb_th + val['Re']*(val['beta'] + 1))
    Ic = val['beta'] * Ib
    Ie = (val['beta'] + 1) * Ib
    
    #KVL 
    #Vcc - Vce - Ie*Re = 0
    #Vce = Vcc - Ie*Re
    Vce = val['Vcc'] - Ie*val['Re']
    
    dc_analysis = {

    #The parallel values for the base
    'Vdc_th': Vdc_th,
    'Rb_th': Rb_th,
    'Vce': Vce,
    'Ib': Ib,
    'Ic': Ic,
    'Ie': Ie,
    }
    
    re = 26e-3 / Ie
    
    
    #Avoc = Vout/Vin = ie*Re/(ie*(re+Re)) = Re/(re+Re)
    Avoc = val['Re']/(re+val['Re'])
    
    #Av = Vout/Vin = (ie(Re || Rl))/(ie*(re+Re||Rl))
    
    #ZinQ = vin/iin = (ie*(re + Re||Rl))/ib = ((beta+1)*ib*(re + Re||Rl))/ib
    #ZinQ = (beta + 1)*(re + Re||Rl)
    ZinQ = (val['beta'] + 1)*(re + 1/(1/val['Re'] + 1/R_Load))
    
    #Zin = Rb_th || ZinQ 
    Zin = 1/(1/Rb_th +1/ZinQ)
    
    #Zoutb = vb/ie = ib*(Rs||Rb)/ie = ib*(Rs||Rb)/((beta+1)*ib) = (Rs||Rb)/(beta+1)
    Zoutb = (1/(1/R_Source+1/Rb_th))/(val['beta']+1)
    
    #ZoutQ = re + Zoutb
    ZoutQ = re + Zoutb
    
    #Zout = Re || ZoutQ
    Zout = 1/(1/val['Re'] + 1/ZoutQ)
    
    ac_analysis = {'re': re, 'Avoc': Avoc}
    amplifier = {'Zin': Zin, 'Zout': Zout, 'gain': Avoc, 'Zoutb':Zoutb, 'ZoutQ':ZoutQ}

    analysis = {'dc': dc_analysis, 'ac': ac_analysis, 'amplifier': amplifier}
    
    
    return analysis

In [17]:

common_collector_amp_circuit = {
    'Vcc': 10, 
    'R1': 16e3,
    'R2': 2.4e3,
    'Re': 100,
    #This circuit is fairly indepedent of beta (100-300)
    'beta': 300,
    'Vbe': 0.7
}

#Loop through betas
R_Load=50
R_Source=1000
analysis = eval_common_collector_amp_circuit(common_collector_amp_circuit,R_Source, R_Load)


print(f"Ib = {analysis['dc']['Ib']:0.3e}A, Ic = {analysis['dc']['Ic']:.3e}A")
print(f"Vth = {analysis['dc']['Vdc_th']:0.2f}V, Rb_th = {analysis['dc']['Rb_th']:.0f} ohms")   
print(f"The Q point is Vce = {analysis['dc']['Vce']:0.2f}V, Ic = {analysis['dc']['Ic']:.3e}A")
print(f"re = {analysis['ac']['re']:0.3e} ohm")
      
# The reason the circuit 
print(f"The predicted open load voltage gain is {analysis['amplifier']['gain']} or {20*np.log10(np.abs(analysis['amplifier']['gain']))} dB")
print(f"Amplifier Zin = {analysis['amplifier']['Zin']:0.1f} ohms, Zout = {analysis['amplifier']['Zout']:0.1f} ohms, Zoutb = {analysis['amplifier']['Zoutb']:0.1f} ohms, ZoutQ = {analysis['amplifier']['ZoutQ']:0.1f} ohms")
      

Ib = 1.878e-05A, Ic = 5.633e-03A
Vth = 1.30V, Rb_th = 2087 ohms
The Q point is Vce = 9.43V, Ic = 5.633e-03A
re = 4.600e+00 ohm
The predicted open load voltage gain is 0.9560188813671946 or -0.3906706065155055 dB
Amplifier Zin = 1764.5 ohms, Zout = 6.4 ohms, Zoutb = 2.2 ohms, ZoutQ = 6.8 ohms


In [ ]:
#